In [2]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.polygons_to_labels import *
from helperToolz.helpsters import *

IACS_path = '/data/Aldhani/eoagritwin/fields/IACS/'

In [ ]:
# set state and year

state = 'Brandenburg'
year = 2019

# check if stuff already in IACS folder??
dum = [file for file in getFilelist(f'{IACS_path}3_Rasterized_lines/', '.tif', deep=True) if 'BRB' in file and str(year) in file and 'lines_touch_false' in file]
if dum:
    rasterized_border_path = dum[0]
else:
    

# 1. rasterize fields (polygons)
# 2 .rasterize borders (after polygons to lines)
# 3. distance to border raster
# 4. IDs 

si


['/data/Aldhani/eoagritwin/fields/IACS/3_Rasterized_lines/2019/GSA-DE_BRB-2019_lines_touch_false.tif']